<h1><center>Machine Learning-basierte Extraktion von phänotypischen Kennzahlen aus UAV-Bildern</center></h1>


<h3><center> Bachelorarbeit von Aymen Benjbara - Matrikel-Nr: 3284477 </center></h3>


### **Projektübersicht**

Dieses Notebook dient als zentrale Schnittstelle für die Nutzung und Bewertung des Projekts. Es ermöglicht eine einfache Handhabung der Funktionen, die in einem externen Skript definiert sind, und bietet eine benutzerfreundliche Umgebung zur Durchführung von Analysen und zur Evaluierung der Ergebnisse.

#### **Struktur**

Die Projektstruktur ist so konzipiert, dass sie eine klare Trennung zwischen der Implementierungslogik und ihrer Anwendung bietet:

- **Extern Skript:** Das Skript 'Bachelorarbeit.py' enthält fast alle Funktionen und Algorithmen, die für das Projekt entwickelt wurden.
- **Jupyter Notebook:** Dient als interaktive Plattform, auf der die Funktionen aus dem Skript aufgerufen und die Ergebnisse visualisiert werden.

#### **Vorteile**

Diese Struktur bietet mehrere Vorteile:

- **Übersichtlichkeit:** Das Notebook bleibt fokussiert auf den Ablauf und die Darstellung der Ergebnisse.
- **Wiederverwendbarkeit:** Funktionen können leicht in anderen Projekten wiederverwendet werden.
- **Wartbarkeit:** Änderungen an Funktionen werden zentral in der Skriptdatei vorgenommen.

#### **Nutzung**

Das Notebook ist so aufgebaut, dass es den Benutzern ermöglicht, mit minimalen Anpassungen verschiedene Aspekte des Projekts zu untersuchen:

1. **Importieren von Bibliotheken und Skripten:** Zu Beginn werden alle notwendigen Bibliotheken und externen Skripte importiert.
3. **EDA, Modelltraining, und -validierung:** Das EDA-Prozess, als auch das Training und die Validierung der Modelle sind im Detail im Notebook  jeden extrahierten Merkmals zu finden. Das Konzept besteht darin, dass die Modelle zur Merkmalsextraktion in separaten Notebooks entwickelt werden, in dem die Datenorganisation und die Trainingsprozesse im Detail erläutert werden. Die Struktur der Arbeit ist wie folgt:  


<table align="center">
    <tr>
        <th>Notebook</th>
        <th>Kennzahlen</th>
    </tr>
    <tr>
        <td>size_estimator.ipynb</td>
        <td>Größe der Pflanze</td>
    </tr>
    <tr>
        <td>leaves_counter.ipynb</td>
        <td>Anzahl der Blättern, Dichte</td>
    </tr>
    <tr>
        <td>height_estimator.ipynb</td>
        <td>Höhe der Pflanze</td>
    </tr>
    <tr>
        <td>head_detection.ipynb</td>
        <td>Blumenkohlkopf Erkennung, Größe des Kopfes</td>
    </tr>
    <tr>
        <td>harvest_classificator.ipynb</td>
        <td>Erntestatus</td>
    </tr>
</table>

 Der Code für das Modelltraining bleibt vollständig ausführbar, um die Flexibilität bei der Untersuchung verschiedener Modelle zu gewährleisten.

#### **Hinweise zur Ausführung**

- Der GPU-Speicher kann manchmal aufgrund der Implementierung der Machine Learning-Modelle überlastet sein, was zum Absturz des Servers führt. In diesem Fall muss der Speicher manuell freigegeben werden, indem folgendes Befehl ausgeführt werden muss: '!nvidia-smi', dann den Prozess, der den meisten Speicher belegt, mit dem Befehl: '!kill PID' beenden. PID bezieht sich auf die Prozess-ID.


## Arbeit

In [ ]:
# Dieser Befehl führt das YOLOv5-Erkennungsskript direkt in einem Jupyter-Notebook aus.
# Er muss als erste Zelle ausgeführt werden, um die YOLO-Umgebung im Notebook korrekt einzurichten.
# Wenn irgendetwas anderes vor dieser Zeile ausgeführt wird, funktioniert der YOLO-Algorithmus möglicherweise nicht korrekt,
# und es könnte notwendig sein, die Umgebung im Notebook zurückzusetzen. Der genaue Grund dafür ist unklar,
# aber es ist wichtig, diesen Schritt zu befolgen, um sicherzustellen, dass YOLO wie erwartet funktioniert.
# Die Ausführung dient nur zur Initialisierung der YOLO-Umgebung.
%run yolov5/detect.py --weights yolov5/runs/train/exp38/weights/best.pt --img 256 --conf 0.7 --source yolov5/test_infer/2021_08_30_post_Def_day4_111.jpg --save-txt --save-conf

Wenn die YOLO-Umgebung wieder integriert werden soll, müssen folgende Befehlszeilen ausgeführt und der Server neu gestartet werden

In [ ]:
# cd yolov5
# %pip install -r requirements.txt  # install dependencies

## Importieren des Hauptskripts

In [ ]:
# Importiert das Skript "Bachelorarbeit".
# Dieses Skript enthält alle Funktionen, die aufgerufen werden, um die Aufgaben direkt über das Notebook auszuführen.
# Dadurch wird der Code minimiert und eine benutzerfreundliche Oberfläche bereitgestellt, mit der die Funktionsweise des Projekts genutzt und getestet werden kann.
import Bachelorarbeit

## Bibliotheken

In [ ]:
# Standardbibliotheken
import os
import glob as gb
import random
import re

# Externe Bibliotheken
import numpy as np
import pandas as pd
import cv2
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.image as mpimg
import matplotlib.patches as patches
import matplotlib.dates as mdates
from PIL import Image, UnidentifiedImageError

# DS und ML Bibliotheken
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

# Bildverarbeitungsbibliotheken
import skimage.feature
# from skimage.feature.texture import greycomatrix, greycoprops

# Visualisierungs- und Anzeigebibliotheken
from IPython.display import Image, display
from IPython.display import set_matplotlib_formats
%matplotlib inline
set_matplotlib_formats('svg')

# Deep-Learning-Bibliotheken
import torch
import torchvision
from torchvision.io import read_image
from torchvision.ops.boxes import masks_to_boxes
from torchvision.transforms.v2 import functional as F
from torchvision.transforms import v2 as T
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from torchvision.utils import draw_bounding_boxes, draw_segmentation_masks
import utils

# Keras 
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.models import load_model

# Gerätekonfiguration für Deep Learning
device = torch.device('cuda')


## Arbeit

In [ ]:
# Diese Funktion Führt das YOLOv5-Detektionsskript aus, um zu überprüfen, ob auf dem Bild eine Pflanze mit Kopf vorhanden ist.
# Sie wird im Notebook und nicht im Skript aufgerufen, da das Skript zur Ausführung des Yolo-Algorithmus nicht durch einen Aufruf vom Bachelorarbeit.py Skript implementiert werden kann.

In [ ]:
def check_head(image_path):
    # Es verwendet eine vortrainierte Gewichte, setzt die Bildgröße auf 256, die Vertrauensschwelle auf 0.7,
    # gibt den Pfad des Bildes an, speichert die Ergebnisse in Textdateien und speichert die Vertrauenswerte.
    %run  yolov5/detect.py --weights yolov5/runs/train/exp38/weights/best.pt --img 256 --conf 0.7 --source {image_path} --save-txt --save-conf
    
    # Ruft den Namen des neuesten Experimentordners innerhalb des YOLOv5-Detektionsverzeichnisses ab.
    exp = Bachelorarbeit.get_highest_exp_file('yolov5/runs/detect/')
    
    # Erstellt den Pfad zum Label-Ordner des aktuellen Experiments.
    label_folder_path = 'yolov5/runs/detect/'+exp+'/labels/'
    
    # Sucht nach allen Textdateien im Label-Ordner, die die Erkennungsergebnisse enthalten.
    txt_files = gb.glob(os.path.join(label_folder_path, '*.txt'))
    
    # Überprüft, ob Textdateien vorhanden sind, was darauf hinweist, dass ein Kopf erkannt wurde.
    IsHead = len(txt_files) > 0
    
    if IsHead:
        print('Die Pflanze hat einen Kopf')
    else:
        print('Die Pflanze hat keinen Kopf')
    
    # Gibt zurück, ob ein Kopf erkannt wurde oder nicht.
    return IsHead


##  Kennzahlen Extraktion aus einzelnen Pflanzen

In [ ]:
# Die Funktion single_plant_features_extraction ist aus dem Modul Bachelorarbeit aufgerufen. 
# Sie ist dafür zuständig, spezifische Merkmale einer Pflanze aus dem gegebenen Bildpfad zu extrahieren. 
# Der Parameter image_path gibt den Pfad zum Bild an, von dem die Merkmale extrahiert werden sollen.   
# Der Parameter hasHead wird durch die Ausführung der Funktion check_head mit demselben Bildpfad als Argument bestimmt.  
# Die Funktion kann nur im Notebook implementiert werden. Sie prüft, ob auf dem Bild ein Pflanzenkopf erkennbar ist, und gibt einen booleschen Wert zurück. 

In [ ]:
#image_path = 'YOUR IMAGE PATH HERE'
image_path = '/home/ab/raid/GrowliFlowerD/Field2/2021_08_30/2021_08_30_Def_day3_204.jpg'
single_plant_features = Bachelorarbeit.single_plant_features_extraction(image_path, hasHead = check_head(image_path))

### Visualisierung des Dataframes, der die extrahierten Kennzahlen enthält

In [ ]:
single_plant_features.head()

### Funktion zum Ausgeben von Merkmalen einer einzelnen Pflanze

In [ ]:
Bachelorarbeit.print_features(single_plant_features)

### Ermittelt das makroskopische Stadium einer Pflanze basierend auf ihren extrahierten Merkmalen.

In [ ]:
Bachelorarbeit.get_macro_stage(single_plant_features)

##  Kennzahlen Extraktion aus einzelnen Pflanzen in Zeitreihen

##### Die Funtion single_plant_time_series ist dafür zuständig, Kennzahlen einer Pflanze durch mehreren Zeitpunkten zu extrahieren. Die Funktion ist im Notebook und nicht im Skript definiert, da sie die Funktion check_head während der Merkmalsextraktion mehrmals aufruft.

<h4><span style="color: red;">Wichtig</span>: Es ist notwendig, den Pfad für die Bilder entsprechend zu bearbeiten. In diesem Beispiel durchläuft die Funktion die Datumsordner und extrahiert Merkmale aus den Bildern, die mit "_Def_day1_5.jpg" enden.</h4>


In [ ]:
def single_plant_time_series(directory_path):
    # Initialisierung von Listen zur Speicherung der Kennzahlen.
    isHarvested_list = []
    predicted_size_list = []
    predicted_height_list = []
    hasHead_list = []
    leaves_count_list = []
    density_rate_list = []  
    head_size_list = []
    # Initialisierung von Zustandsvariablen.
    isHarvested = False
    wasHarvested = False
    prev_density_rate = None
    
    # Sortieren und Durchlaufen der Datumsordner im angegebenen Verzeichnis.
    dates = sorted(os.listdir(directory_path))
    # Festlegen von Datumsangaben, die übersprungen werden sollen.
    skip_dates = ['2020_10_29', '_post', '_pre']   
    for date in dates:
        # Überspringen bestimmter Datumsangaben basierend auf den definierten Kriterien.
        if date in skip_dates or '_post' in date or '_pre' in date:
            continue
        # Erstellen des Bildpfades.
        image_path = f'{directory_path}/{date}/{date}_Def_day1_7.jpg'  #  !!!!!!!!! Den Pfad entsprechend bearbeiten !!!!!!!!!
        # Überprüfen, ob das Bild existiert.
        if not os.path.exists(image_path):
            print(f'No image for {date}')
            continue
        
        # Speichern des Datums als Dateiname.
        filename = date
        
        # Überprüfen, ob ein Kopf auf dem Bild erkennbar ist.
        hasHead = check_head(image_path)
        hasHead_list.append((filename, hasHead))
        
        # Wenn ein Kopf erkennbar ist, wird die Größe des Kopfes bestimmt. Andernfalls wird überprüft, ob die Pflanze geerntet wurde.
        if hasHead:
            head_size = Bachelorarbeit.head_size_function(image_path)
        else:
            head_size = 0
            isHarvested = Bachelorarbeit.harvest_classif_model(image_path)
            
        head_size_list.append((filename, head_size))
     
        # Überprüfung, ob die Pflanze geerntet wurde, basierend auf vorherigen Kennzahlen und dem aktuellen Zustand.
        if not isHarvested and not hasHead:
            if prev_density_rate is not None and density < prev_density_rate / 2:
                isHarvested = True

        if wasHarvested: 
            isHarvested = True
        
        # Wenn festgestellt wird, dass die Pflanze geerntet wurde, werden keine weiteren Kennzahlen extrahiert und alle Listen mit Standardwerten gefüllt.
        if isHarvested:
            print('The plant is harvested, no features to extract')
            isHarvested_val = 1
            wasHarvested = True 
            
            isHarvested_list.append((filename, isHarvested_val))
            predicted_size_list.append((filename, 0))
            leaves_count_list.append((filename, 0))
            density_rate_list.append((filename, 0))
            predicted_height_list.append((filename, 0))
            hasHead_list.append((filename, False))
            head_size_list.append((filename, 0))
            continue
        else:
            isHarvested_val = 0
            wasHarvested = False
            
            isHarvested_list.append((filename, isHarvested_val))
       
        isHarvested_list.append((filename, isHarvested_val))
    
        # Extraktion und Visualisierung der Bounding-Boxen.
        Bachelorarbeit.get_bounding_boxes_plot(image_path)
        
        # Extraktion weiterer Kennzahlen wie Größe, Blattanzahl, Dichte und Höhe der Pflanze.
        predicted_size = Bachelorarbeit.predict_size(image_path)
        predicted_size_list.append((filename, predicted_size))
  
        output_image, num_boxes_leaves, leaves_number, density  = Bachelorarbeit.leaves_count(image_path)
        leaves_count_list.append((filename, leaves_number))
        
        density_rate_list.append((filename, density))
        prev_density_rate = density
        
        predicted_height = Bachelorarbeit.predict_height_value(image_path)
        predicted_height_list.append((filename, predicted_height))

    # Zusammenführung der gesammelten Kennzahlen in einem DataFrame und Rückgabe dieses DataFrames.
    features_dataframe = Bachelorarbeit.features_dataframe(isHarvested_list, predicted_size_list, predicted_height_list, hasHead_list, leaves_count_list, density_rate_list, head_size_list)
    return features_dataframe

In [ ]:
#directory_path = 'YOUR DIRECTORY PATH HERE'
directory_path = 'raid/GrowliFlowerD/Field2/'  
image_features = single_plant_time_series(directory_path)

In [ ]:
image_features.head(15)

## Visulisierung der extrahierten Kennzahlen in Zeitreihen

### Planzengröße

In [ ]:
Bachelorarbeit.plant_size_plot(image_features)

### Planzenhöhe 

In [ ]:
Bachelorarbeit.plant_height_plot(image_features)

### Anzahl der Blättern

In [ ]:
Bachelorarbeit.leaves_count_plot(image_features)

### Dichtrate

In [ ]:
Bachelorarbeit.density_rate_plot(image_features)

### Größe des Blumenkohlkopfs

In [ ]:
Bachelorarbeit.head_size_plot(image_features)

### Visulisierung multipler Kennzahlen

##### Die zu visualisierenden Kennzahlen können verändert werden, aber sie müssen Drei bleiben.

In [ ]:
# Liste der Kennzahlen: 'plant_size', 'plant_height', 'leaves_count', 'density_rate', 'head_size'
Bachelorarbeit.plot_features(image_features, 'plant_size' , 'leaves_count', 'density_rate')

##  Kennzahlen Extraktion aus mehreren Pflanzen

##### Definiert eine Funktion zur Extraktion von Merkmalen mehrerer Pflanzen aus einem Verzeichnis.

In [ ]:
def multiple_plants_features_extraction(directory_path):
    # Initialisiert eine Liste zur Speicherung von DataFrames.
    df_list = []

    # Durchläuft alle Dateien im Verzeichnis.
    for filename in os.listdir(directory_path):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            # Ermittelt den vollständigen Pfad des Bildes.
            image_path = os.path.join(directory_path, filename)

            # Extrahiert die Kennzahlen für eine einzelne Pflanze.
            single_plant_features_dataframe = Bachelorarbeit.single_plant_features_extraction(image_path, hasHead = check_head(image_path))

            # Fügt das DataFrame der einzelnen Pflanzenkennzahlen zur Liste hinzu.
            df_list.append(single_plant_features_dataframe)

    # Fügt alle DataFrames in der Liste zu einem einzigen DataFrame zusammen.
    features_dataframe = pd.concat(df_list, ignore_index=True)

    # Gibt das zusammengefügte DataFrame zurück.
    return features_dataframe

In [ ]:
# Die Bilder müssen in einem einzigen Ordner angeordnet sein
#directory_path = 'YOUR DIRECTORY PATH HERE'
directory_path = 'raid/multi_extrac'
multiple_plants_features = multiple_plants_features_extraction(directory_path)

In [ ]:
multiple_plants_features.head(15)

### Visualisierung der extrahierten Kennzahlen von mehreren Pflanzen

In [ ]:
Bachelorarbeit.multiple_plants_plot(multiple_plants_features, 'head_size')

##  Kennzahlen Extraktion aus mehreren Pflanzen in Zeitreihen

#####  Die Funktion multiple_plants_time_series ist darauf ausgelegt, eine Zeitreihe von Merkmalen für mehrere Pflanzen zu erstellen, basierend auf Bildern, die in einem gegebenen Basisverzeichnis geordnet sind.

In [ ]:
def multiple_plants_time_series(base_directory_path):
    # Initialisiert ein Dictionary, um für jede Pflanze ein DataFrame zu speichern.
    plants_dataframes = {}

    # Ermittelt alle Verzeichnisse, die Daten enthalten.
    date_directories = sorted([d for d in os.listdir(base_directory_path) if os.path.isdir(os.path.join(base_directory_path, d))])
    
    for date_dir in date_directories:
        # Überspringt unerwünschte Datumsangaben.
        # Bilder im Datum 2020_10_29 und 2021_08_19 sind unvollständig 
        skip_dates = ['2020_10_29', '2021_08_19', '_post', '_pre']    
        if date_dir in skip_dates:
            continue
        
        # Pfad zum aktuellen Datumsverzeichnis.
        current_date_path = os.path.join(base_directory_path, date_dir)
        
        # Listet Bilddateien im aktuellen Datumsverzeichnis auf.
        image_files = [f for f in os.listdir(current_date_path) if f.endswith('.jpg')]
        
        for image_file in image_files:
            try:
                # Erstellt den vollständigen Pfad zur Bilddatei.
                image_path = os.path.join(current_date_path, image_file)
                
                # Extrahiert den Pflanzennamen (Dateiname ohne Endung).
                plant_name = os.path.splitext(image_file)[0]
                plant_name_parts = plant_name.split('_')[-2:]
                plant_name = '_'.join(plant_name_parts)
                
                # Initialisiert eine Liste für diese Pflanze, falls sie noch nicht existiert.
                if plant_name not in plants_dataframes:
                    plants_dataframes[plant_name] = []

                # Extrahiert erneut den Pflanzennamen und initialisiert ein DataFrame, falls dies noch nicht geschehen ist.
                plant_name = os.path.splitext(image_file)[0]
                plant_name = plant_name.split('_')[-2:]
                plant_name = '_'.join(plant_name)
                if plant_name not in plants_dataframes:
                    plants_dataframes[plant_name] = pd.DataFrame(columns=[
                        'date', 'isHarvested', 'plant_size', 'plant_height', 
                        'hasHead', 'leaves_count', 'density_rate', 'head_size'
                    ])

                isHarvested = False    
                hasHead = check_head(image_path)
                # Bestimmt die Kopfgröße, wenn vorhanden.
                if hasHead:
                    head_size = Bachelorarbeit.head_size_function(image_path)
                else:
                    head_size = 0
                    isHarvested = Bachelorarbeit.harvest_classif_model(image_path)
                # Überprüft, ob die Pflanze geerntet wurde.
                if isHarvested:
                    print('Die Pflanze wurde geerntet, keine Kennzahlen zu extrahieren')
                    isHarvested = 1
                    predicted_size = 0
                    leaves_number = 0
                    density = 0
                    predicted_height = 0
                else:
                    isHarvested = 0
                    
                    Bachelorarbeit.get_bounding_boxes_plot(image_path)
                    # Vorhersage der Pflanzengröße.
                    predicted_size =  Bachelorarbeit.predict_size(image_path)
                    # Zählung der Blätter und Dichte.
                    output_image, num_boxes_leaves, leaves_number, density  =  Bachelorarbeit.leaves_count(image_path)
                    # Vorhersage der Pflanzenhöhe.
                    predicted_height =  Bachelorarbeit.predict_height_value(image_path)
                
                # Erstellt ein Dictionary mit den extrahierten Kennzahlen.

                features = {
                'date': date_dir,
                'isHarvested': isHarvested,
                'plant_size': predicted_size,
                'plant_height': predicted_height,
                'hasHead': int(hasHead),
                'leaves_count': leaves_number,
                'density_rate': density,
                'head_size': head_size,
                }
                
                # Fügt die Kennzahlen der entsprechenden Liste hinzu.
                plants_dataframes[plant_name].append(features)
                
            except Exception as e:
                print(f"Beim Verarbeiten von {image_file} ist ein Fehler aufgetreten: {e}")

    # Konvertiert Listen von Kennzahlen in DataFrames.
    for plant_name, features_list in plants_dataframes.items():
        plants_dataframes[plant_name] = pd.DataFrame(features_list)

    # Transformiert die DataFrames für jede Pflanze.
    for plant_name, df in plants_dataframes.items():
        plants_dataframes[plant_name] =  Bachelorarbeit.transform_dataframe(df)

    return plants_dataframes

In [ ]:
# Für die Ausführung der Funktion ist die Einhaltung einer spezifischen Ordnerstruktur essentiell. 
# Ein Basisverzeichnis soll diverse Unterverzeichnisse für Bildmaterial umfassen, wobei jedes dieser Unterverzeichnisse nach dem Datum benannt wird, welches es repräsentiert. 
# Innerhalb jedes Datumsverzeichnisses müssen sich Bilder befinden, die am entsprechenden Datum aufgenommen wurden. 
# Zudem ist es erforderlich, dass alle Datumsverzeichnisse eine identische Anzahl von Bildern beinhalten.

#base_directory_path = 'YOUR DIRECTORY PATH HERE'
base_directory_path = 'raid/series_extraction_2'
all_plants_dataframes = multiple_plants_time_series(base_directory_path)

### Visualisierung der extrahierten Kennzahlen von mehreren Pflanzen in Zeitreihen

#####  Diese Funktion plottet die extrahierten Merkmale verschiedener Pflanzenbilder, indem das Merkmal angegeben wird, das visualisiert werden soll 

In [ ]:
# Liste der Kennzahlen: 'plant_size', 'plant_height', 'leaves_count', 'density_rate', 'head_size'
Bachelorarbeit.plot_time_series_features(all_plants_dataframes, 'leaves_count')

<h1><center> <font color="red">Ende</center></h1>